In [ ]:
# EXTRACCION

In [1]:
!pip install spotipy python-dotenv
import base64
from requests import post,get
import json
import numpy as np
import pandas as pd

In [2]:
CLIENT_ID=
CLIENT_SECRET=
def get_token():
    auth_string = CLIENT_ID+":"+CLIENT_SECRET 
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"

    headers = {
        "Authorization" : "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type":"client_credentials"}
    result = post(url,headers=headers,data=data)
    json_result = json.loads(result.content)
    token = json_result
    return token["access_token"]
def get_auth_header(token):
    return {"Authorization": "Bearer "+ token}

In [3]:
def search_for_playlist(token,playlist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query=f"?q={playlist_name}&type=playlist&limit=1"
    query_url = url + query
    result = get(query_url,headers=headers)
    json_result = json.loads(result.content)
    return  json_result

def get_playlist(token, playlist_id):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)
    return json_result

def get_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}"  # ← Esto está bien
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)
    return json_result


def get_features(token,songs_id):
    url=f"https://api.spotify.com/v1/audio-features?ids={songs_id}"
    headers = get_auth_header(token)
    result = get(url,headers=headers)
    json_result = json.loads(result.content)
    return json_result

In [4]:
token = get_token()

playlist_info=search_for_playlist(token,"billboard hot 100")

playlist_name=playlist_info["playlists"]["items"][0]["name"]
playlist_description=playlist_info["playlists"]["items"][0]["description"]
playlist_id=playlist_info["playlists"]["items"][0]["id"]

print(playlist_name)
print(playlist_description)
print(playlist_id)

playlist_json=get_playlist(token,playlist_id)

Billboard Hot 100
The official <a href="http://www.billboard.com/charts/hot-100">Billboard Hot 100</a> features this week's most popular songs across all genres, ranked by radio airplay monitored by Nielsen BDS, download sales tracked by Nielsen SoundScan and streaming activity data provided by leading online music services. For more information go to <a href="http://Billboard.com">Billboard.com</a>.
6UeSakyzhiEt4NB3UAd6NQ


In [5]:
token = get_token()

playlist_info=search_for_playlist(token,"billboard hot 100")

playlist_name=playlist_info["playlists"]["items"][0]["name"]
playlist_description=playlist_info["playlists"]["items"][0]["description"]
playlist_id=playlist_info["playlists"]["items"][0]["id"]

print(playlist_name)
print(playlist_description)
print(playlist_id)

playlist_json=get_playlist(token,playlist_id)

Billboard Hot 100
The official <a href="http://www.billboard.com/charts/hot-100">Billboard Hot 100</a> features this week's most popular songs across all genres, ranked by radio airplay monitored by Nielsen BDS, download sales tracked by Nielsen SoundScan and streaming activity data provided by leading online music services. For more information go to <a href="http://Billboard.com">Billboard.com</a>.
6UeSakyzhiEt4NB3UAd6NQ


In [6]:
columns=["top",
        "track_name",
        "track_popularity",
        "duration_ms",
        "artist_name",
        "artist_genres",
        "artist_popularity",
        "feats",
        "explicit",
        "album",
        "type",
        "relase_date",
        "track_id",
        "artist_id"]
df_playlist= pd.DataFrame(columns=columns)

In [7]:
track_info=playlist_json["tracks"]["items"]
feats=[]

for i in range(len(playlist_json["tracks"]["items"])):
      data={"top":i+1,
          "track_name": track_info[i]["track"]["name"],
          "track_popularity": track_info[i]["track"]["popularity"],

          "artist_name":track_info[i]["track"]["artists"][0]["name"],
          "artist_genres":"",
          "duration_ms":track_info[i]["track"]["duration_ms"],
          "artist_popularity":"",
          "feats":"",
          "explicit":track_info[i]["track"]["explicit"],
          "album":track_info[i]["track"]["album"]["name"],
          "type":track_info[i]["track"]["album"]["album_type"],
          "relase_date":track_info[i]["track"]["album"]["release_date"],
          "track_id":track_info[i]["track"]["id"],
          "artist_id":track_info[i]["track"]["artists"][0]["id"]
          }

      artist_info=get_artist(token,data["artist_id"])
      data["artist_popularity"]=artist_info["popularity"]

      if len(artist_info["genres"])==1:

        data["artist_genres"]=artist_info["genres"][0]

      elif len(artist_info["genres"])>1:
        generes=artist_info["genres"]
        data["artist_genres"]="|".join(generes)

      else:
          data["artist_genres"]=np.nan


      if len(track_info[i]["track"]["artists"])>1:
        for j in range(len(track_info[i]["track"]["artists"])-1):
            feats.append(track_info[i]["track"]["artists"][j+1]["name"])

        data["feats"]="|".join(feats)
      else:
        data["feats"]=np.nan
      feats=[]

      df_playlist = pd.concat([df_playlist, pd.DataFrame([data])], ignore_index=True)

In [8]:
print(df_playlist.head())
print(df_playlist.shape)

# Exportar CSV con ruta completa
df_playlist.to_csv("BillboardHot100.csv", index=False)

# Confirmar que se ha guardado
import os
print("Archivo guardado:", os.path.isfile("BillboardHot100.csv"))

  top                      track_name track_popularity duration_ms  \
0   1                        Ordinary               96      186964   
1   2  What I Want (feat. Tate McRae)               75      184517   
2   3                    Just In Case               83      166206   
3   4              A Bar Song (Tipsy)               88      171291   
4   5                 I'm The Problem               79      177811   

     artist_name artist_genres artist_popularity       feats explicit  \
0    Alex Warren           NaN                90         NaN    False   
1  Morgan Wallen       country                95  Tate McRae    False   
2  Morgan Wallen       country                95         NaN    False   
3      Shaboozey           NaN                82         NaN     True   
4  Morgan Wallen       country                95         NaN    False   

                album    type relase_date                track_id  \
0            Ordinary  single  2025-02-07  6qqrTXSdwiJaq8SO0X2lSe   
1 